In [14]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import random
from selenium.webdriver.support import expected_conditions as EC
import time
from random import randint
from thefuzz import process
import json
# Set the path to your downloaded Edge WebDriver executable

driver = webdriver.Edge()

# Initialize the Edge WebDriver
# driver = webdriver.Edge(service=edge_service)
# driver.implicitly_wait(randint(1,3))



In [15]:
secrets_file = r"C:\Users\localadmin\Desktop\myprojects\linkedinsucks\secrets.json"
with open(secrets_file, 'r') as file:
    data = json.load(file)
print(data)

{'linkedin_loginid': 'ganeshrajk007@gmail.com', 'linkedin_password': 'Linkedin@531', 'OPENAI_KEY': '', 'HUGGINGFACE_KEY': '', 'LANGCHAIN_KEY': ''}


In [34]:
questions_file = r"C:\Users\localadmin\Desktop\myprojects\linkedinsucks\secrets.json"
with open(questions_file, 'r') as file:
    questions_dict = json.load(file)
print(questions_dict)

{'linkedin_loginid': 'ganeshrajk007@gmail.com', 'linkedin_password': 'Linkedin@531', 'OPENAI_KEY': '', 'HUGGINGFACE_KEY': '', 'LANGCHAIN_KEY': ''}


In [16]:
#user details
# define system variables for the url 


linkedin_url = "https://www.linkedin.com/"
password = data['linkedin_password']
username = data['linkedin_loginid']
role = "Software Developer"
location = "United States"

In [17]:
def open_linkedin_and_signin():
    driver.get(linkedin_url)
    driver.maximize_window()
    signin_page=driver.find_element("xpath", ".//a[contains(text(), 'Sign in') and @href]")
    signin_page.click()
    username_field = driver.find_element("xpath" , ".//input[@id = 'username']")
    password_field = driver.find_element("xpath" , ".//input[@id = 'password']")
    submit_button = driver.find_element("xpath" , ".//button[@type = 'submit']")
    username_field.send_keys(username)
    password_field.send_keys(password)
    submit_button.click()

In [18]:
def update_questions(path_json, questions_dict):
    with open(path_json, 'r') as file:
    # Load the JSON data
        data = json.load(file)
    file.close()

    data.update(questions_dict)

    with open(path_json, 'w') as file:
        json.dump(questions_dict, file)
    file.close()



In [19]:
# defining job search filters  

def easy_apply():
    easy_apply_button = driver.find_element( "xpath" , "//button[contains(@aria-label ,'Easy Apply filter.')]") 
    easy_apply_button.click()

In [20]:
#opening linkedin 
def get_url(url):
    driver.get(url)
    driver.maximize_window()
    
    
def sign_in(username, password):
    signin_page = driver.find_element("xpath", ".//a[contains(text(), 'Sign in') and @href]")
    signin_page.click()
    username_field = driver.find_element("xpath" , ".//input[@id = 'username']")
    password_field = driver.find_element("xpath" , ".//input[@id = 'password']")
    submit_button = driver.find_element("xpath" , ".//button[@type = 'submit']")
    
    username_field.send_keys(username)
    password_field.send_keys(password)
    submit_button.click()

    
def job_search_and_filters():
    get_url("https://www.linkedin.com/jobs")
    role_input = driver.find_element("xpath" , ".//input[@class = 'jobs-search-box__text-input jobs-search-box__keyboard-text-input jobs-search-global-typeahead__input']")
    role_input.send_keys(role)
    time.sleep(1)
    location_input = driver.find_element("xpath" , ".//input[@aria-label = 'City, state, or zip code' and @autocomplete=  'address-level2']")
    location_input.send_keys(location)
    #submit
    role_input.send_keys(Keys.ENTER)
    time.sleep(2)
    easy_apply()

In [21]:
time_dict = {
    "Any time": 0,
    "Past month": 1,
    "Past week": 2,
    "Past 24 hours": 3
}


exp_dict = {
    "Internship": 0,
    "Entry level": 1,
    "Associate": 2,
    "Mid-Senior level": 3,
    "Director": 4,
    "Executive": 5
}

salary_dict = {
    "$40,000+": 0,
    "$60,000+": 1,
    "$80,000+": 2,
    "$100,000+": 3,
    "$120,000+": 4,
    "$140,000+": 5,
    "$160,000+": 6,
    "$180,000+": 7,
    "$200,000+": 8
}

workmode_dict = {
    "On-site": 0,
    "Remote": 1,
    "Hybrid": 2
}


In [22]:
def check_no_matches():
    no_matches  = driver.find_elements('xpath' , ".//h1[@class='t-24 t-black t-normal text-align-center']")
    if no_matches:
        return True
def reset_filters():
    reset_button = driver.find_element('xpath',".//button[@aria-label='Reset applied filters']")
    reset_button.click()



In [23]:
def date_filter( user_date):
    date_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'timePostedRange']  " )
    date_options = date_button.find_elements( "xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    date_button.click()
    label = date_options[time_dict[user_date]].find_element("xpath" , ".//label")
    label.click()
    results = date_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()



def exp_filter( exp_selected):
    exp_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'experience']  " )
    exp_options = exp_button.find_elements("xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    exp_button.click()
    for exp in exp_selected:
        label = exp_options[exp_dict[exp]].find_element("xpath" , ".//label")
        label.click()
    results = exp_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()


def salary_filter(  base_salary):
    salary_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'salaryBucketV2']  " )
    salary_options = salary_button.find_elements("xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    salary_button.click()
    label = salary_options[salary_dict[base_salary]].find_element("xpath" , ".//label")
    label.click()
    results = salary_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()


def workmode_filter(  workmode_selected):
    workmode_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'workplaceType']  " )
    hybrid_options = workmode_button.find_elements("xpath" , ".//li[@class = 'search-reusables__collection-values-item'] ")
    workmode_button.click()
    for mode in workmode_selected:
        label = hybrid_options[workmode_dict[mode]].find_element("xpath" , ".//label")
        label.click()
    results = workmode_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()




def company_filter(companies):

    company_button = driver.find_element("xpath" , ".//div[@data-basic-filter-parameter-name = 'company']  ")
    company_button.click()
    input_field = driver.find_element("xpath", ".//input[@aria-label='Add a company']")
    for company in companies:
    
        # input_field = driver.find_element("xpath", ".//input[@aria-label='Add a company']")
        input_field.clear()
        input_field.send_keys(f"{company}")
        time.sleep(0.5)
        input_field.send_keys(Keys.ARROW_DOWN)
        input_field.send_keys(Keys.ENTER)
    results = company_button.find_element("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml2'] ")
    results.click()
    time.sleep(2)

    if check_no_matches():
        reset_filters()



In [24]:
# date_params = 'Past month'
# exp_param = ['Internship' , 'Entry level']
# salary_param = "$80,000+"
# workmode_param= ['On-site' , 'Hybrid']
# companies = ['blackrock' , 'amazon']

In [25]:
# date_filter(date_params)
# exp_filter(exp_param)
# salary_filter(salary_param)
# workmode_filter(workmode_param)
# company_filter(companies=companies)

In [26]:
def theresubmit(driver):
    buttons = driver.find_elements( "xpath" , ".//button[@aria-label = 'Submit application']") 
    return len(buttons) and True

def get_submit(driver):
    button = driver.find_element( "xpath" , ".//button[@aria-label = 'Submit application']") 
    button.click()



def there_resume(form):
    resumes = form.find_elements("xpath" ,".//div[contains(@class, 'ui-attachment jobs-document-upload-redesign-card__container')]")
    return len(resumes) and True
    

def select_resume( number):
    resumes = driver.find_elements("xpath" ,".//div[contains(@class, 'ui-attachment jobs-document-upload-redesign-card__container')]")
    driver.execute_script("arguments[0].click();", resumes[number]) 
    resumes[number].click()   
    


def click_next():
    button = driver.find_element("xpath" , ".//button[@aria-label = 'Continue to next step' or @aria-label = 'Review your application']")
    button.click()


In [27]:
def radio_select(radio_buttons):

    for radio_button in radio_buttons:


        question = radio_button.find_element(By.XPATH, ".//legend/span[@data-test-form-builder-radio-button-form-component__title]").text
        option_elements = radio_button.find_elements(By.XPATH, ".//div[@data-test-text-selectable-option]")
        options = []
        
        for option in option_elements:
            options.append(  option.find_element(By.XPATH, ".//label").text)
        print(question, options)
        answer = input("name: ")
        
        answer_match = process.extractOne(answer, options)[0] 

        questions_dict[question] = answer_match
        selected_option = option_elements[options.index(answer_match)].find_element(By.XPATH, ".//input[@type='radio']")
        driver.execute_script("arguments[0].click();", selected_option)
        #driver.execute_script("arguments[0].click();", selected_option)
    

def dropdown_select(dropdowns):
    # Extract the question
    for dropdown_element in dropdowns:
        question = dropdown_element.find_element(By.XPATH, ".//label/span[@aria-hidden='true']").text
        
        # Extract the options
        options = [option.text for option in dropdown_element.find_elements(By.XPATH, ".//select/option")]
        
        # Print the question and options
        print("Question:", question)
        print("Options:", options)
        answer = input("Enter your name: ")
        # Find the best match for the answer
        answer_match = process.extractOne(answer, options)[0]
        questions_dict[question] = answer_match
        # Select the matched option using Select
        select_element = dropdown_element.find_element(By.XPATH, ".//select[contains(@id, 'text-entity-list-form-component-formElement')]")
        select = Select(select_element)
        select.select_by_visible_text(answer_match)




def fill_single_line_text_field(text_fields):
    # Extract the question

    for text_field in text_fields:
        question = text_field.find_element(By.XPATH, ".//label").text
        
        # Print the question
        print("Question:", question)
        answer = input("Enter your name: ")
        questions_dict[question] = answer
        # Fill the answer in the input field
        input_field = text_field.find_element(By.XPATH, ".//input[@type='text']")
        input_field.send_keys(answer)

def fill_multiline_text_field(text_fields):
    for text_field in text_fields:
    # Extract the question
        question = text_field.find_element(By.XPATH, ".//label").text
        answer = input("Enter your name: ")
        # Print the question
        print("Question:", question)
        questions_dict[question] = answer
        # Fill the answer in the textarea field
        textarea_field = text_field.find_element(By.XPATH, ".//textarea")
        textarea_field.send_keys(answer)






def checkbox_select(fieldset_elements):
    # Extract the question
    for fieldset_element in fieldset_elements:
        question = fieldset_element.find_element(By.XPATH, ".//legend/div[@data-test-checkbox-form-title='true']").text
        answers = input("Enter your name: ")
        # Extract the options
        option_elements = fieldset_element.find_elements(By.XPATH, ".//div[@data-test-text-selectable-option]")
        options = [option.find_element(By.XPATH, ".//label").text for option in option_elements]
        
        # Print the question and options
        print("Question:", question)
        print("Options:", options)
        questions_dict[question]  = answers
        # Select the matched options
        for answer in answers:
            answer_match = process.extractOne(answer, options)[0]
            option_index = options.index(answer_match)
            checkbox = option_elements[option_index].find_element(By.XPATH, ".//input[@type='checkbox']")
            if not checkbox.is_selected():
                checkbox.click()




def autofill_answer(autofill_fields):
    for autofill_field in autofill_fields:
        input_field = autofill_field.find_element("xpath" , ".//input[@aria-autocomplete='list']")
        answer = input("Enter your name: ")
        questions_dict[question] = answer
        input_field.send_keys(answer)
        time.sleep(1.0)
        input_field.send_keys(Keys.DOWN)
        input_field.send_keys(Keys.ENTER)

    

In [28]:
def apply_job(job_card):
    job_card.click()
    try:
        easy_apply_button = driver.find_element("xpath" , ".//button[contains(@aria-label , 'Easy Apply to')]")
        easy_apply_button.click()
    except Exception as e:
        return
    print("clicked easy apply")
    
    filledresume = False
    applying = True

    while applying:
        if theresubmit(driver):
            get_submit(driver)
            break

        
        apply = driver.find_element("xpath" , ".//div[@class='jobs-easy-apply-content']")
        form = apply.find_element("xpath", ".//form")

        if not filledresume and there_resume(form):
            select_resume(0)
            filledresume = True

        form = driver.find_element("xpath", ".//form")
        radios = form.find_elements("xpath" , ".//fieldset[@data-test-form-builder-radio-button-form-component = 'true']" )
        if radios:
            radio_select(radios )

        form = driver.find_element("xpath", ".//form")
        text_dropdowns = form.find_elements("xpath" ,".//div[@data-test-text-entity-list-form-component='']")
        if text_dropdowns:
            dropdown_select(text_dropdowns)

        form = driver.find_element("xpath", ".//form")
        singleline_textfields = form.find_elements("xpath" , ".//div[@data-test-single-line-text-form-component='']")
        if singleline_textfields:
            fill_single_line_text_field(singleline_textfields)

        form = driver.find_element("xpath", ".//form")
        multiline_fields = form.find_elements("xpath" , ".//div[@data-test-multiline-text-form-component='']")
        if multiline_fields:
            fill_multiline_text_field(multiline_fields)

        form = driver.find_element("xpath", ".//form")
        checkboxes = form.find_elements("xpath" , ".//fieldset[@data-test-checkbox-form-component='true']") 
        if checkboxes:
            checkbox_select(checkboxes)
        autofills = form.find_elements("xpath" , ".//input[@aria-autocomplete='list']")
        if autofills:
            autofill_answer(autofills)

        click_next()
        


    

    time.sleep(2)
    closingpopup = True
    while closingpopup: 
        
        print("finding exit")
        exit_buttons = driver.find_elements("xpath" , ".//button[@aria-label='Dismiss' ]")
        if exit_buttons:
            exit_buttons = driver.find_elements("xpath" , ".//button[@aria-label='Dismiss' ]")
            exit_buttons[0].click()
            print("exited")
            return
        


In [29]:
# def apply_job(job_card):
    
#     job_card.click()

    
#     easy_apply_button = driver.find_element("xpath", ".//button[contains(@aria-label , 'Easy Apply to')]")
#     easy_apply_button.click()
#     print("clicked easy apply")
    
#     isform = True
#     fillresume = False
    
#     while isform:

#         if theresubmit(driver):
#             get_submit(driver)


        
#         apply = driver.find_element("xpath" , ".//div[@class='jobs-easy-apply-content']")
#         try:
#             form = apply.find_element("xpath", ".//form")
#         except Exception as e:
#             isform = False
        
#         if not fillresume:
            
#             if there_resume(form):
#                 print("reached resume")
#                 resumes = form.find_elements("xpath" ,".//div[contains(@class, 'ui-attachment jobs-document-upload-redesign-card__container')]")
#                 driver.execute_script("arguments[0].click();", resumes[0]) 
#                 resumes[0].click()
#                 # enter_resume(form, 0)
#                 fillresume = True

          
#         fields = form.find_elements("xpath" , "//div[contains(@class, 'jobs-easy-apply-form-section__grouping')]")
#         radios = form.find_elements("xpath" , ".//fieldset[@data-test-form-builder-radio-button-form-component = 'true']" )
#         text_dropdowns = form.find_elements("xpath" ,".//div[@data-test-text-entity-list-form-component='']")
#         singleline_textfields = form.find_elements("xpath" , ".//div[@data-test-single-line-text-form-component='']")
#         multiline_fields = form.find_elements("xpath" , ".//div[@data-test-multiline-text-form-component='']")
#         checkboxes = form.find_elements("xpath" , ".//fieldset[@data-test-checkbox-form-component='true']") 
#         autofills = form.find_elements("xpath" , ".//input[@aria-autocomplete='list']")

#         for radio in radios:
#             radio_select(radio)
#         for textdropdown in text_dropdowns:
#             dropdown_select(textdropdown)
#         for singletext in singleline_textfields:
#             fill_single_line_text_field(singletext)
#         for multitext in multiline_fields:
#             fill_multiline_text_field(multitext)
#         for checkbox in checkboxes:
#             checkbox_select(checkbox)
#         for autofill in autofills:
#             autofill_answer(autofill)
      
        



#         find_next(form)

        
 


    
#     exit_buttons = driver.find_elements("xpath" , ".//button[@class = 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view mlA block' and .//span[text()='Done' ]]")
#     print("reacjhed exot")
#     if exit_buttons:
#         exit_buttons[0].click()

    


In [30]:
def iterator( driver):
    currentpage = 1
    nextpageexists = True
    
    while nextpageexists:
        
        search_results = driver.find_element("xpath" , ".//div[contains(@class, 'jobs-search-results-list')]")
        list_of_jobs = search_results.find_elements( "xpath" , "//li[@data-occludable-job-id]")
        
        attempt_job = 0
        while attempt_job < 3:
            try:
                driver.execute_script("arguments[0].scrollIntoView();", list_of_jobs[-1])
                
                attempt_job = 4
            except Exception as e:
                attempt_job += 1
        if attempt_job == 3:
            return
       
        for job in list_of_jobs:
           
            job.click() 
            time.sleep(0.2)
            apply_job(job)
            path_json = 'QnA.json'
            
            update_questions(path_json, questions_dict)
            
        
        

        
        attempt_job = 0
        while attempt_job < 3:
            try:
                
                driver.execute_script("arguments[0].click();", list_of_jobs[-1])
                attempt_job = 4
            except Exception as e:
                attempt_job += 1
        if attempt_job == 3:
            return
        
        # next_pages = driver.find_elements("xpath", f"//button[@aria-label = 'Page {currentpage + 1}']")
        currentpage+=1


        attempts = 0
        while attempts < 5:
            try:
                driver.find_element("xpath", f"//button[@aria-label = 'Page {currentpage + 1}']").click()
                nextpageexists = True
                attempts = 5
            except Exception as e:
                attempts += 1
                nextpageexists = False

        if nextpageexists:
            print("moving to next page")
            
            
        else:
            print("ending function")
            return
    return
    

            

In [31]:
open_linkedin_and_signin()
job_search_and_filters()

In [33]:

iterator(driver)

clicked easy apply
Question: Phone country code
Options: ['Select an option', 'United States (+1)', 'Afghanistan (+93)', 'Aland Islands (+358)', 'Albania (+355)', 'Algeria (+213)', 'American Samoa (+1)', 'Andorra (+376)', 'Angola (+244)', 'Anguilla (+1)', 'Antarctica (+0)', 'Antigua and Barbuda (+1)', 'Argentina (+54)', 'Armenia (+374)', 'Aruba (+297)', 'Australia (+61)', 'Austria (+43)', 'Azerbaijan (+994)', 'Bahamas (+1)', 'Bahrain (+973)', 'Bangladesh (+880)', 'Barbados (+1)', 'Belarus (+375)', 'Belgium (+32)', 'Belize (+501)', 'Benin (+229)', 'Bermuda (+1)', 'Bhutan (+975)', 'Bolivia (+591)', 'Bosnia and Herzegovina (+387)', 'Botswana (+267)', 'Bouvet Island (+0)', 'Brazil (+55)', 'British Indian Ocean Territory (+246)', 'Brunei Darussalam (+673)', 'Bulgaria (+359)', 'Burkina Faso (+226)', 'Burundi (+257)', 'Cambodia (+855)', 'Cameroon (+237)', 'Canada (+1)', 'Cape Verde (+238)', 'Caribbean Nations (+0)', 'Cayman Islands (+1)', 'Central African Republic (+236)', 'Chad (+235)', 'Chi

Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']


NameError: name 'questions_dict' is not defined